__Problem 3__ The file IBM-MSFT-HAS.csv on Gauchospace provides the closing daily prices for IBM, MSFT (Microsoft) and HAS (Hasbro) over the past year. We will use this data to empirically estimate market portfolios within the Markowitz framework.

a) Consider the last 6 months: from Nov 1, 2017 to Apr 30, 2018. Using this subset of the data, estimate the monthly average return, variance and covariance among the three assets. To obtain monthly returns, estimate the daily returns and rescale by a factor of 20 (=number of trading days per month).

In [1]:
import pandas as pd
import numpy as np 
import itertools 
import timeit
from scipy.stats import binom
%matplotlib inline


data = pd.read_csv('IBM-MSFT-HAS.csv')
data['Date'] = pd.to_datetime(data['Date'])
start_date = '10/31/2017'; end_date = '4/30/2018'
subset = data.loc[(data['Date'] > start_date) & (data['Date'] <= end_date)]
subset.set_index('Date', inplace = True)

returns = subset.pct_change()[1:]

print('monthly average returns = \n%s'%(20*returns.mean(axis = 0), ))
print('\nmonthly variance = \n%s'%((20*returns).var()))
print('\nmonthly covariance matrix = \n%s'%((20*returns).cov(), ))

monthly average returns = 
IBM    -0.004541
MSFT    0.023690
HAS     0.000764
dtype: float64

monthly variance = 
IBM     0.086371
MSFT    0.116455
HAS     0.120840
dtype: float64

monthly covariance matrix = 
           IBM      MSFT       HAS
IBM   0.086371  0.056839  0.016176
MSFT  0.056839  0.116455  0.022038
HAS   0.016176  0.022038  0.120840



b) Using above information, compute the weights of the Market portfolio, assuming interest rate of $r = 0.02$. Report also the risk and return of $K_M$.

In [2]:
from numpy.linalg import det, inv 

def muX(w, mm):
    return np.dot(np.transpose(w), np.transpose(mm))

def sigmaX(w, co):
    w = np.transpose(w)
    return np.sqrt(np.dot(np.dot(w, co), np.transpose(w)))

r = (1.02)**(1/12) - 1

mu = np.array(20*returns[1:].mean(axis = 0))
C = np.array((20*returns).cov())

uVec = np.transpose(np.array([1, 1, 1]))
wMarket = np.dot((mu - r*uVec), inv(C))/sum(np.dot((mu - r*uVec), inv(C)))
mu_Mk = muX(np.transpose(wMarket), mu); sigma_Mk = sigmaX(np.transpose(wMarket), C)

print('3b) The market portfolio weight is %s; its associated'%(wMarket,))
print('expected return is %s and standard deviation is %s.'%(mu_Mk, sigma_Mk))

3b) The market portfolio weight is [-9.03707    10.55650116 -0.51943116]; its associated
expected return is 0.26747251169224334 and standard deviation is 3.021506674082392.


c) Repeat above for the dataset from May 1, 2017 to Oct 31, 2017. How similar are the weights/risk/return profile of KM compared to part b)?


In [3]:
start_date = '4/30/2017'; end_date = '10/31/2017'
subset1 = data.loc[(data['Date'] > start_date) & (data['Date'] <= end_date)]
subset1.set_index('Date', inplace = True)

returns = subset1.pct_change()[1:]
mu = np.array(20*returns[1:].mean(axis = 0))
C = np.array((20*returns).cov())

uVec = np.transpose(np.array([1, 1, 1]))
wMarket = np.dot((mu - r*uVec), inv(C))/sum(np.dot((mu - r*uVec), inv(C)))
mu_Mk = muX(np.transpose(wMarket), mu); sigma_Mk = sigmaX(np.transpose(wMarket), C)

print('3c) The market portfolio weight is %s; its associated'%(wMarket,))
print('expected return is %s and standard deviation is %s.'%(mu_Mk, sigma_Mk))
print('Compared to part b, the weights are quite different in terms of ratio. However, the return and risk profile seem to be adjusted accordingly.')

3c) The market portfolio weight is [-0.0527191   1.53341932 -0.48070022]; its associated
expected return is 0.05197235963521041 and standard deviation is 0.3079243991428129.
Compared to part b, the weights are quite different in terms of ratio. However, the return and risk profile seem to be adjusted accordingly.



d) Repeat once more for the full dataset over the entire last year.

In [4]:
start_date = '4/30/2017'; end_date = '4/30/2018'
subset2 = data.loc[(data['Date'] > start_date) & (data['Date'] <= end_date)]
subset2.set_index('Date', inplace = True)

returns = subset2.pct_change()[1:]
mu = np.array(20*returns[1:].mean(axis = 0))
C = np.array((20*returns).cov())

uVec = np.transpose(np.array([1, 1, 1]))
wMarket = np.dot((mu - r*uVec), inv(C))/sum(np.dot((mu - r*uVec), inv(C)))
mu_Mk = muX(np.transpose(wMarket), mu); sigma_Mk = sigmaX(np.transpose(wMarket), C)

print('3c) The market portfolio weight is %s; its associated'%(wMarket,))
print('expected return is %s and standard deviation is %s.'%(mu_Mk, sigma_Mk))

3c) The market portfolio weight is [-2.48388543  4.81629912 -1.33241369]; its associated
expected return is 0.1462475688360461 and standard deviation is 1.2478715579317763.


__Problem 4__ Consider a binomial tree with $u = 1.1, d = 0.92, S_0 = 20$ and $r = 0.04$. The physical probability is $p = 0.5$. We consider an exponential utility maximization problem. Let $U(x) = −2e^{−x/2}$.

a) Find the optimal terminal wealth $X^∗_N$ that maximizes E[$U(X_N$)] as a function of number of periods $N$ and initial wealth $X_0$. Write out explicitly $X^∗_3$ for the three-period model $N = 3$ and initial wealth $X_0 = 10$ (i.e. give the numerical value of $X_3(HHH)$, $X_3(HHT)$, et cetera.) Compute the resulting expected utility $v(X_0) = E[U(X^∗_3$)] at $X_0 = 10$.

In [5]:
S0 = 20; u = 1.1; r = 0.04; d = 0.92; n = 3; M = 3; p = 0.5 
q = (1+r-d)/(u-d)

def permgrid(n, u, d):
    inds = np.indices((2,) * n)
    inds = inds.reshape(n, -1).T
    inds = d*inds
    return np.where(inds == 0, u, inds)

def binomialPath(M):
    PQ = permgrid(M, u, d)
    S = np.zeros((2**n, n+1))
    S[:, 0] = S0 
    
    for i in range(n):
        S[:, i+1] = S[:, i]*PQ[:, i]
              
    return S 

def expandgrid(*itrs):
    
    product = list(itertools.product(*itrs))
    
    return np.transpose(np.array([[x[i] for x in product] for i in range(len(itrs))]))
    
    
def optimal_w(N, x0):
    w = expandgrid(*[[0, 1]]*N); pd = 1
    ZN = np.zeros(2**N); XiN = np.zeros(2**N)
    Q = np.zeros(2**N); VT = np.zeros(2**N)
    
    for i in range(2**N):
        H = sum(w[i, :] == 1)
        T = sum(w[i, :] == 0)
        ZN[i] = (q/p)**H*((1-q)/(1-p))**T
        XiN[i] = ZN[i]*(1+r)**(-N)
        Q[i] = q**H*(1-q)**T
        pd = pd*(XiN[i])**Q[i]
    
    l = np.exp(-0.5*x0*(1+r)**N)/pd; UT = 0 
    for i in range(2**N):
        VT[i] = -2*np.log(l*XiN[i])
        UT = UT + (p**N)*(-2*np.exp(-VT[i]/2))
    
    return VT, UT

S = binomialPath(M)

VT, UT = optimal_w(3, 10)

print('Optimal terminal wealth is %s.'%(VT,))
print('Expected utility is %s.' %UT)  

Optimal terminal wealth is [14.02122872 12.63493436 12.63493436 11.24864    12.63493436 11.24864
 11.24864     9.86234564].
Expected utility is -0.006090215230644724.


b) [276 students only] Starting from your answer in part (a), compute the optimal investment portfolios along the scenario $THH$: report $\Delta_0$, $\Delta_1(T)$, $\Delta_2(TH)$ the corresponding bond investments $B$’s and the wealth $X_0 = 10, X_1(T), X_2(T H), X_3(THH)$.

In [6]:
def xi(xi_T, xi_H):
    return (xi_H*q + xi_T*(1-q))/(1+r)

X2HH = xi(*VT[6:8]); X2HT = xi(*VT[4:6])
X2TH = xi(*VT[2:4]); X2TT = xi(*VT[0:2])

print('X2(HH) = %s' %(X2HH))
print('X2(HT) = %s' %(X2HT))
print('X2(TH) = %s' %(X2TH))
print('X2(TT) = %s' %(X2TT))

X2(HH) = 9.927349768512894
X2(HT) = 11.260325115743555
X2(TH) = 11.260325115743555
X2(TT) = 12.593300462974215


In [7]:
def delta(x1, x2, s1, s2):
    return (x1 - x2)/(s1 - s2)
X1H = xi(X2HT, X2HH) 
X1T = xi(X2TT, X2TH)

print('X1(H) = %s' %(X1H))
print('X1(T) = %s' %(X1T))

X0 = xi(X1T, X1H)
print('X0 = %s' %(X0))
print('delta2(TH) = %s' %delta(VT[3], VT[2], S[4, 3], S[5, 3]))
print('delta1(T) = %s' %delta(X2TH, X2TT, S[5, 2], S[6, 2]))
print('delta0 = %s' %delta(X1H, X1T, S[1, 1], S[6, 1]))

X1(H) = 9.972764311785046
X1(T) = 11.25447137642991
X0 = 10.0
delta2(TH) = -0.380515580017536
delta1(T) = -0.40246840194162414
delta0 = -0.35602974017912925


c) [276 students only] Repeat for $U(x) = −5e^{−x/5}$ – how do the new investment portfolios compare to the answers in (b)? Discuss in terms of the risk-aversion parameter $\gamma$ in $U(x) = −\frac{1}{\gamma}e^{−\gamma x}$.

In [8]:
def optimal_w(N, x0):
    w = expandgrid(*[[0, 1]]*N); pd = 1
    ZN = np.zeros(2**N); XiN = np.zeros(2**N)
    Q = np.zeros(2**N); VT = np.zeros(2**N)
    
    for i in range(2**N):
        H = sum(w[i, :] == 1)
        T = sum(w[i, :] == 0)
        ZN[i] = (q/p)**H*((1-q)/(1-p))**T
        XiN[i] = ZN[i]*(1+r)**(-N)
        Q[i] = q**H*(1-q)**T
        pd = pd*(XiN[i])**Q[i]
    
    l = np.exp(-0.2*x0*(1+r)**N)/pd; UT = 0 
    for i in range(2**N):
        VT[i] = -5*np.log(l*XiN[i])
        UT = UT + (p**N)*(-5*np.exp(-VT[i]/5))
    
    return VT, UT

VT, UT = optimal_w(3, 10)

print('Optimal terminal wealth is %s.'%(VT,))
print('Expected utility is %s.' %UT)  

X2HH = xi(*VT[6:8]); X2HT = xi(*VT[4:6])
X2TH = xi(*VT[2:4]); X2TT = xi(*VT[0:2])

print('X2(HH) = %s' %(X2HH))
print('X2(HT) = %s' %(X2HT))
print('X2(TH) = %s' %(X2TH))
print('X2(TT) = %s' %(X2TT))

X1H = xi(X2HT, X2HH) 
X1T = xi(X2TT, X2TH)

print('X1(H) = %s' %(X1H))
print('X1(T) = %s' %(X1T))

X0 = xi(X1T, X1H)
print('X0 = %s' %(X0))
print('delta2(TH) = %s' %delta(VT[3], VT[2], S[4, 3], S[5, 3]))
print('delta1(T) = %s' %delta(X2TH, X2TT, S[5, 2], S[6, 2]))
print('delta0 = %s' %delta(X1H, X1T, S[1, 1], S[6, 1]))

Optimal terminal wealth is [18.18011181 14.7143759  14.7143759  11.24864    14.7143759  11.24864
 11.24864     7.7829041 ].
Expected utility is -0.4447739406806444.
X2(HH) = 8.594374421282229
X2(HT) = 11.926812789358884
X2(TH) = 11.926812789358884
X2(TT) = 15.25925115743554
X1(H) = 9.331910779462609
X1(T) = 12.536178441074778
X0 = 9.999999999999998
delta2(TH) = -0.9512889500438415
delta1(T) = -1.0061710048540622
delta0 = -0.8900743504478253


Compared to part (b), the expected utility is less; however, the optimal wealth is comparably higher on average than the optimal wealth presented in part (b). It seems that $\gamma$ correlates with the optimal terminal wealth values.